In [1]:
import librosa
from IPython.display import Audio
import numpy as np
import pandas as pd
import os
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import torchaudio
from torch.utils.data import DataLoader
import audiomentations
from sklearn.preprocessing import LabelEncoder

In [26]:
class CONFIG:
    ramdom_seed= 42
    
    sample_rate = 32000
    duration=10
    audio_len = duration*sample_rate
    target_length = 
    
    img_size = [128,256]
    num_classes = 264
    batch_size = 32
    
    nfft = 2028
    window = 2048
    hop_length = audio_len // (img_size[1] - 1)
    fmin = 20
    fmax = 16000
    normalize = True

In [17]:
meta_train2023.loc[meta_train2023['primary_label'].str.contains('crefra')]

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename,filepath,class
5529,crefra2,[],"['adult', 'call', 'sex uncertain']",-0.5283,31.0002,Ortygornis sephaena,Crested Francolin,Niall Perrins,Creative Commons Attribution-NonCommercial-Sha...,3.0,http://xeno-canto.org/667443,crefra2/XC667443.ogg,./train_audio/crefra2/XC667443.ogg,75


In [3]:
meta_train2023['primary_label'].value_counts()

wlwwar     500
eaywag1    500
thrnig1    500
comsan     500
barswa     500
          ... 
brtcha1      1
crefra2      1
afpkin1      1
whctur2      1
yebsto1      1
Name: primary_label, Length: 264, dtype: int64

In [14]:
meta_train2023

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename,filepath,class
0,abethr1,[],['song'],4.3906,38.2788,Turdus tephronotus,African Bare-eyed Thrush,Rolf A. de By,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/128013,abethr1/XC128013.ogg,./train_audio/abethr1/XC128013.ogg,0
1,abethr1,[],['call'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/363501,abethr1/XC363501.ogg,./train_audio/abethr1/XC363501.ogg,0
2,abethr1,[],['song'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/363502,abethr1/XC363502.ogg,./train_audio/abethr1/XC363502.ogg,0
3,abethr1,[],['song'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/363503,abethr1/XC363503.ogg,./train_audio/abethr1/XC363503.ogg,0
4,abethr1,[],"['call', 'song']",-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://www.xeno-canto.org/363504,abethr1/XC363504.ogg,./train_audio/abethr1/XC363504.ogg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16936,yewgre1,[],[''],-1.2502,29.7971,Eurillas latirostris,Yellow-whiskered Greenbul,András Schmidt,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://xeno-canto.org/703472,yewgre1/XC703472.ogg,./train_audio/yewgre1/XC703472.ogg,263
16937,yewgre1,[],[''],-1.2489,29.7923,Eurillas latirostris,Yellow-whiskered Greenbul,András Schmidt,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/703485,yewgre1/XC703485.ogg,./train_audio/yewgre1/XC703485.ogg,263
16938,yewgre1,[],[''],-1.2433,29.7844,Eurillas latirostris,Yellow-whiskered Greenbul,András Schmidt,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/704433,yewgre1/XC704433.ogg,./train_audio/yewgre1/XC704433.ogg,263
16939,yewgre1,[],[''],0.0452,36.3699,Eurillas latirostris,Yellow-whiskered Greenbul,Lars Lachmann,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/752974,yewgre1/XC752974.ogg,./train_audio/yewgre1/XC752974.ogg,263


In [2]:
# Load metadata
meta_train2023 = pd.read_csv("train_metadata.csv")
meta_train2023.head(5)
meta_train2023['filepath'] = './train_audio/'+meta_train2023['filename']
# Encode Labels
le = LabelEncoder()
meta_train2023['class'] = le.fit_transform(meta_train2023['primary_label'])

In [16]:
au ,sr = librosa.load(meta_train2023['filepath'].iloc[5529] ,sr=None)
Audio(au,rate=sr)

In [27]:
def crop_or_pad(audio, target_len = CONFIG.target_len
                , pad_mode='constant', take_first=True):
    audio_len = audio.shape[0]
    diff_len = abs(target_len - audio_len)
    if audio_len < target_len:
        pad1 = torch.randint(high=diff_len, size=[], dtype=torch.int32)
        pad2 = diff_len - pad1
        audio = torch.nn.functional.pad(audio, (pad1.item(), pad2.item()), mode=pad_mode)
    elif audio_len > target_len:
        if take_first:
            audio = audio[:target_len]
        else:
            idx = torch.randint(high=diff_len, size=[], dtype=torch.int32)
            audio = audio[idx.item(): (idx.item() + target_len)]
    return torch.reshape(audio, [target_len])

AttributeError: type object 'CONFIG' has no attribute 'target_len'

In [38]:
# padding
import torch.nn.functional as F
t4d = torch.ones(3, 3, 4, 2)
t1d = t4d.reshape(-1)
print(t4d.reshape(-1))
print(t1d.shape)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
torch.Size([72])


In [39]:
p1d = (1, 1) # pad last dim by 1 on each side
out = F.pad(t1d, p1d, "constant", 0)  # effectively zero padding
print(out)
print(out.size())

tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0.])
torch.Size([74])
